### In this notebook we will make the data preprocessing for the MeteoLibre dataset.

1. First the h5 dataset (not a lot to do here the data is already processed and ready to be used). 

The list of files already exist nothing to be done in reality

2. Second the ground station dataset

3. The ammping creation to get the correct position in EPSG

4. The two bufr file to be preprocess (and created via the bufr_preprocessing.py script)



In [1]:
import pandas as pd
import os
import datetime

## 2. Ground station

In [2]:
# 2. we first take a look at the data.gouv files to get the url of the historical data


file_ressources = "../data/datagouv/2ad89e9d0b014ad0fc3b605dc69b9d41.parquet"

ressources = pd.read_parquet(file_ressources)
ressources.head()

,dataset.id,dataset.title,dataset.slug,dataset.url,dataset.organization,dataset.organization_id,dataset.license,dataset.private,dataset.archived,id,...,checksum.type,checksum.value,created_at,modified,downloads,harvest.created_at,harvest.modified_at,schema_name,schema_version,preview_url
0,6793192601c56fbc19bed9b0,Plus hautes eaux connues reconstituées sur la ...,plus-hautes-eaux-connues-reconstituees-sur-la-...,http://www.data.gouv.fr/fr/datasets/plus-haute...,DREAL Centre-Val de Loire,55b23b0e88ee3839da3ca288,Licence Ouverte / Open Licence version 2.0,False,False,fe6486be-3ccf-494b-b8f3-d3786e995959,...,None,None,2025-01-24T04:37:58.841000,2025-01-24T04:37:58.841000,0,None,None,None,None,False
1,6793192601c56fbc19bed9b0,Plus hautes eaux connues reconstituées sur la ...,plus-hautes-eaux-connues-reconstituees-sur-la-...,http://www.data.gouv.fr/fr/datasets/plus-haute...,DREAL Centre-Val de Loire,55b23b0e88ee3839da3ca288,Licence Ouverte / Open Licence version 2.0,False,False,3768cc5d-fbed-4bfa-9bda-29f6805941b7,...,None,None,2025-01-24T04:37:58.841000,2025-01-24T04:37:58.841000,0,None,None,None,None,False
2,6793192601c56fbc19bed9af,Plus hautes eaux connues reconstituées sur la ...,plus-hautes-eaux-connues-reconstituees-sur-la-...,http://www.data.gouv.fr/fr/datasets/plus-haute...,DREAL Centre-Val de Loire,55b23b0e88ee3839da3ca288,Licence Ouverte / Open Licence version 2.0,False,False,cebe02ba-28b5-41cc-9864-7fd095357894,...,None,None,2025-01-24T04:37:58.568000,2025-01-24T04:37:58.568000,0,None,None,None,None,False
3,6793192601c56fbc19bed9af,Plus hautes eaux connues reconstituées sur la ...,plus-hautes-eaux-connues-reconstituees-sur-la-...,http://www.data.gouv.fr/fr/datasets/plus-haute...,DREAL Centre-Val de Loire,55b23b0e88ee3839da3ca288,Licence Ouverte / Open Licence version 2.0,False,False,8a0d615f-eb33-46ca-aaf0-e7049601cb93,...,None,None,2025-01-24T04:37:58.568000,2025-01-24T04:37:58.568000,0,None,None,None,None,False
4,6793192601c56fbc19bed9ae,Plus hautes eaux connues reconstituées sur la ...,plus-hautes-eaux-connues-reconstituees-sur-la-...,http://www.data.gouv.fr/fr/datasets/plus-haute...,DREAL Centre-Val de Loire,55b23b0e88ee3839da3ca288,Licence Ouverte / Open Licence version 2.0,False,False,f63964ce-77a4-4d69-b884-e132ff6b8103,...,None,None,2025-01-24T04:37:58.269000,2025-01-24T04:37:58.269000,0,None,None,None,None,False


In [3]:
id_datastations = "6569b4473bedf2e7abad3b72"
ressources_meteofrance = ressources[ressources["dataset.id"] == id_datastations]

In [4]:
ressources_meteofrance["description"].dropna().values

array(['Données horaires pour le département 01, sur la période 1850-1859',
       'Données horaires pour le département 01, sur la période 1890-1899',
       'Données horaires pour le département 01, sur la période 1900-1909',
       ...,
       'Données horaires pour le département 988, sur la période 2020-2023',
       'Données horaires pour le département 988, sur la période 2024-2025',
       'Données horaires pour le département 99, sur la période 1980-1989'],
      dtype=object)

In [5]:
# drop false
values = ressources_meteofrance["description"].dropna().str.contains("2024-2025").values

idx = ressources_meteofrance["description"].dropna()[values].index

In [6]:
data_to_download = ressources_meteofrance.loc[idx]

In [7]:
data_to_download[["description", "url"]].head().values


array([['Données horaires pour le département 01, sur la période 2024-2025',
        'https://object.files.data.gouv.fr/meteofrance/data/synchro_ftp/BASE/HOR/H_01_latest-2024-2025.csv.gz'],
       ['Données horaires pour le département 02, sur la période 2024-2025',
        'https://object.files.data.gouv.fr/meteofrance/data/synchro_ftp/BASE/HOR/H_02_latest-2024-2025.csv.gz'],
       ['Données horaires pour le département 03, sur la période 2024-2025',
        'https://object.files.data.gouv.fr/meteofrance/data/synchro_ftp/BASE/HOR/H_03_latest-2024-2025.csv.gz'],
       ['Données horaires pour le département 03, sur la période 2024-2025 pour les stations de type 5',
        'https://object.files.data.gouv.fr/meteofrance/data/synchro_ftp/BASE/HOR_COMP/H-COMP_03_latest-2024-2025.csv.gz'],
       ['Données horaires pour le département 04, sur la période 2024-2025',
        'https://object.files.data.gouv.fr/meteofrance/data/synchro_ftp/BASE/HOR/H_04_latest-2024-2025.csv.gz']],
      dtype

In [8]:
data_to_download[['id', 'url', 'title',
       'description', 'filetype', 'type', 'format', 'mime', 'filesize']]

,id,url,title,description,filetype,type,format,mime,filesize
191337,5c6e8f51-0d49-4379-9b40-31ba904dff4d,https://object.files.data.gouv.fr/meteofrance/...,HOR_departement_01_periode_2024-2025,"Données horaires pour le département 01, sur l...",remote,main,csv.gz,None,4271656.0
191348,214e7a8e-ce86-47b7-90d0-5a34f1a3190b,https://object.files.data.gouv.fr/meteofrance/...,HOR_departement_02_periode_2024-2025,"Données horaires pour le département 02, sur l...",remote,main,csv.gz,None,4243129.0
191368,33ba5fe2-df2e-49cf-963c-2517b43ca5a0,https://object.files.data.gouv.fr/meteofrance/...,HOR_departement_03_periode_2024-2025,"Données horaires pour le département 03, sur l...",remote,main,csv.gz,None,5235451.0
191369,1b934e8e-3df4-4745-9322-58bb62a6d68d,https://object.files.data.gouv.fr/meteofrance/...,HOR-COMP_departement_03_periode_2024-2025,"Données horaires pour le département 03, sur l...",remote,main,csv.gz,None,250185.0
191383,7d9c4e73-68b8-4e93-97a6-0f94feb21e50,https://object.files.data.gouv.fr/meteofrance/...,HOR_departement_04_periode_2024-2025,"Données horaires pour le département 04, sur l...",remote,main,csv.gz,None,5694456.0
...,...,...,...,...,...,...,...,...,...
193033,b4fd99f0-82f4-4c64-b613-9d64f21bac4c,https://object.files.data.gouv.fr/meteofrance/...,HOR_departement_984_periode_2024-2025,"Données horaires pour le département 984, sur ...",remote,main,csv.gz,None,3454512.0
193043,6824721d-69fa-428c-8cd8-6eee45dbda21,https://object.files.data.gouv.fr/meteofrance/...,HOR_departement_985_periode_2024-2025,"Données horaires pour le département 985, sur ...",remote,main,csv.gz,None,2134359.0
193050,54e0d850-087c-4a46-98d0-03c9b945f56b,https://object.files.data.gouv.fr/meteofrance/...,HOR_departement_986_periode_2024-2025,"Données horaires pour le département 986, sur ...",remote,main,csv.gz,None,1195987.0
193068,221c1468-708a-4465-af41-b397bbae701a,https://object.files.data.gouv.fr/meteofrance/...,HOR_departement_987_periode_2024-2025,"Données horaires pour le département 987, sur ...",remote,main,csv.gz,None,13027328.0


In [ ]:
# loop over data_to_download and download the data from dataset.url col
for url in data_to_download["url"]:
    !wget {url} -O ../data/groundstations/{url.split("/")[-1]}

--2025-02-23 15:33:03--  https://object.files.data.gouv.fr/meteofrance/data/synchro_ftp/BASE/HOR/H_01_latest-2024-2025.csv.gz
Resolving object.files.data.gouv.fr (object.files.data.gouv.fr)... 91.134.21.160, 162.19.98.210, 91.134.21.159, ...
Connecting to object.files.data.gouv.fr (object.files.data.gouv.fr)|91.134.21.160|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4613956 (4,4M) [application/gzip]
Saving to: ‘../data/groundstations/H_01_latest-2024-2025.csv.gz’

../data/groundstati 100%[===================>]   4,40M  20,5MB/s    in 0,2s    

2025-02-23 15:33:03 (20,5 MB/s) - ‘../data/groundstations/H_01_latest-2024-2025.csv.gz’ saved [4613956/4613956]

--2025-02-23 15:33:04--  https://object.files.data.gouv.fr/meteofrance/data/synchro_ftp/BASE/HOR/H_02_latest-2024-2025.csv.gz
Resolving object.files.data.gouv.fr (object.files.data.gouv.fr)... 91.134.21.160, 162.19.98.210, 91.134.21.159, ...
Connecting to object.files.data.gouv.fr (object.files.data.gouv.fr

### 3. Concatenation of data

In [ ]:
## now for each files in data/groundstations we unzip the .gz files read it, create a parquet file
list_files = os.listdir("../data/groundstations")

# only .gz file
#list_files = [f for f in list_files if f.endswith(".gz")]
list_files = [f for f in list_files if f.endswith(".csv")]

# we loop over the file and unzip it read it save it into parquet format and delete the .gz file
for file in list_files:
    try:
        print(file)
        #!gzip -d ../data/groundstations/{file}
        
        # remove the last .gz 
        # file = file[:-3]

        # read the csv file
        df = pd.read_csv(f"../data/groundstations/{file}", sep=";")
        
        # save it into parquet format
        df.to_parquet(f"../data/groundstations_parquet/{file.split('.')[0]}.parquet")
        
        # delete the .gz file
        # os.remove(f"../data/groundstations/{file}")
    except Exception as e:
        
        print(e)

In [ ]:
data_parquet.head()

In [2]:

dir_ = "../data/groundstations_parquet"

# we list all the parquet file in ../data/groundstations_parquet
list_parquet = os.listdir(dir_)

# we take only the parquet file that start with H
list_parquet = [file for file in list_parquet if ".parquet" in file]

list_df = []

for file in list_parquet:
       
       print(file)
       
       data_parquet = pd.read_parquet(dir_ + "/" + file)

       describe_info = data_parquet.describe()

       # we keep only the columns with more that half non NaN values
       idx_columns = describe_info.loc["count"] > len(data_parquet) / 2

       columns_taken = idx_columns.index[idx_columns.values]

       columns_taken = ['NUM_POSTE', 'LAT', 'LON', 'ALTI', 'AAAAMMJJHH', 'RR1', 'QRR1', 'FF',
              'QFF', 'DD', 'QDD', 'FXY', 'QFXY', 'DXY', 'QDXY', 'HXY', 'QHXY', 'FXI',
              'QFXI', 'DXI', 'QDXI', 'HXI', 'QHXI', 'FXI3S', 'QFXI3S', 'HFXI3S',
              'QHFXI3S', 'T', 'QT', 'TN', 'QTN', 'HTN', 'QHTN', 'TX', 'QTX', 'HTX',
              'QHTX', 'DG', 'QDG']

       custom_df = data_parquet[list(columns_taken) + ["NOM_USUEL"]]
       
       # save the custom_df 
       custom_df.to_parquet("../data/groundstations_filter/{}".format(file))

       list_df.append(custom_df)


H_34_latest-2024-2025.parquet
H_03_latest-2024-2025.parquet
H_87_latest-2024-2025.parquet
H-COMP_22_latest-2024-2025.parquet
H-COMP_15_latest-2024-2025.parquet
H-COMP_91_latest-2024-2025.parquet
H_69_latest-2024-2025.parquet
H_07_latest-2024-2025.parquet
H_30_latest-2024-2025.parquet
H_83_latest-2024-2025.parquet
H-COMP_48_latest-2024-2025.parquet
H-COMP_11_latest-2024-2025.parquet
H-COMP_26_latest-2024-2025.parquet
H_65_latest-2024-2025.parquet
H_52_latest-2024-2025.parquet
H-COMP_44_latest-2024-2025.parquet
H_38_latest-2024-2025.parquet
H_56_latest-2024-2025.parquet
H_61_latest-2024-2025.parquet
H-COMP_19_latest-2024-2025.parquet
H-COMP_40_latest-2024-2025.parquet
H-COMP_77_latest-2024-2025.parquet
H_988_latest-2024-2025.parquet
H-COMP_69_latest-2024-2025.parquet
H-COMP_07_latest-2024-2025.parquet
H-COMP_30_latest-2024-2025.parquet
H_48_latest-2024-2025.parquet
H_11_latest-2024-2025.parquet
H_26_latest-2024-2025.parquet
H_95_latest-2024-2025.parquet
H-COMP_34_latest-2024-2025.parquet

In [3]:
df_total = pd.concat(list_df)

# get proper format for AAAAMMJJHH
df_total["datetime"] = pd.to_datetime(df_total["AAAAMMJJHH"], format="%Y%m%d%H", errors="raise")

df_total.index = df_total["datetime"]

In [4]:
df_total.to_parquet("../data/groundstations_filter/total.parquet")

In [2]:
df_total = pd.read_parquet("../data/groundstations_filter/total.parquet")

In [8]:
# Select columns with 'float64' dtype  
float64_cols = list(df_total.select_dtypes(include='float64'))

# The same code again calling the columns
df_total[float64_cols] = df_total[float64_cols].astype('float32')

In [9]:
df_total.memory_usage()

Index         204124976
NUM_POSTE     204124976
LAT           102062488
LON           102062488
ALTI          204124976
AAAAMMJJHH    204124976
RR1           102062488
QRR1          102062488
FF            102062488
QFF           102062488
DD            102062488
QDD           102062488
FXY           102062488
QFXY          102062488
DXY           102062488
QDXY          102062488
HXY           102062488
QHXY          102062488
FXI           102062488
QFXI          102062488
DXI           102062488
QDXI          102062488
HXI           102062488
QHXI          102062488
FXI3S         102062488
QFXI3S        102062488
HFXI3S        102062488
QHFXI3S       102062488
T             102062488
QT            102062488
TN            102062488
QTN           102062488
HTN           102062488
QHTN          102062488
TX            102062488
QTX           102062488
HTX           102062488
QHTX          102062488
DG            102062488
QDG           102062488
NOM_USUEL     204124976
datetime      20

In [10]:
df_total.to_parquet("../data/groundstations_filter/total_float32.parquet")

In [ ]:
from pyproj import Transformer 

positions_stations = df[["LAT", "LON"]].drop_duplicates()

EPSG = "32630"

# now we want to apply the transformer to the positions of the stations
transformer = Transformer.from_proj("EPSG:4326", f"EPSG:{EPSG}")

positions_stations_transformed = transformer.transform(positions_stations["LAT"], positions_stations["LON"])

# create the dataframe with the mapping 
df_mapping = pd.DataFrame({
    "LAT": positions_stations["LAT"],
    "LON": positions_stations["LON"],
    "LAT_transformed": positions_stations_transformed[0],
    "LON_transformed": positions_stations_transformed[1]
})

# save it somewhere
df_mapping.to_parquet("scripts/positions_stations_transformed.parquet")

In [ ]:


df_total.loc[2025022100]

